# Web Scraping with Beautiful Soup

* * * 

### Icons used in this notebook
🔔 **Question**: A quick question to help you understand what's going on.<br>
🥊 **Challenge**: Interactive exercise. We'll work through these in the workshop!<br>
⚠️ **Warning**: Heads-up about tricky stuff or common mistakes.<br>
💡 **Tip**: How to do something a bit more efficiently or effectively.<br>
🎬 **Demo**: Showing off something more advanced – so you know what Python can be used for!<br>

### Learning Objectives
1. [Reflection: To Scape Or Not To Scrape](#when)
2. [Extracting and Parsing HTML](#extract)
3. [Scraping the Illinois General Assembly](#scrape)

<a id='when'></a>

# To Scrape Or Not To Scrape

When we'd like to access data from the web, we first have to make sure if the website we are interested in offers a Web API. Platforms like Twitter, Reddit, and the New York Times offer APIs. **Check out D-Lab's [Python Web APIs](https://github.com/dlab-berkeley/Python-Web-APIs) workshop if you want to learn how to use APIs.**

However, there are often cases when a Web API does not exist. In these cases, we may have to resort to web scraping, where we extract the underlying HTML from a web page, and directly obtain the information we want. There are several packages in Python we can use to accomplish these tasks. We'll focus two packages: Requests and Beautiful Soup.

Our case study will be scraping information on the [state senators of Illinois](http://www.ilga.gov/senate), as well as the [list of bills](http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True) each senator has sponsored. Before we get started, peruse these websites to take a look at their structure.

## Installation

We will use two main packages: [Requests](http://docs.python-requests.org/en/latest/user/quickstart/) and [Beautiful Soup](http://www.crummy.com/software/BeautifulSoup/bs4/doc/). Go ahead and install these packages, if you haven't already:

In [ ]:
# Instalación del paquete 'requests' para realizar solicitudes HTTP a páginas web.

In [1]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Instalación del paquete 'beautifulsoup4' para analizar (parsear) el contenido HTML de páginas web.

In [2]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


We'll also install the `lxml` package, which helps support some of the parsing that Beautiful Soup performs:

In [ ]:
# Instalación del parser 'lxml', que es rápido y eficiente para trabajar con HTML y XML.

In [3]:
%pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import required libraries
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import time

<a id='extract'></a>

# Extracting and Parsing HTML 

In order to succesfully scrape and analyse HTML, we'll be going through the following 4 steps:
1. Make a GET request
2. Parse the page with Beautiful Soup
3. Search for HTML elements
4. Get attributes and text of these elements

## Step 1: Make a GET Request to Obtain a Page's HTML

We can use the Requests library to:

1. Make a GET request to the page, and
2. Read in the webpage's HTML code.

The process of making a request and obtaining a result resembles that of the Web API workflow. Now, however, we're making a request directly to the website, and we're going to have to parse the HTML ourselves. This is in contrast to being provided data organized into a more straightforward `JSON` or `XML` output.

se actualiza el link actual para obtener los datos de los miembros ya que la pagina original ha cambiado

In [ ]:
# Realizamos una solicitud HTTP GET a la página del Senado de Illinois.
# Luego guardamos el contenido HTML de la respuesta en la variable 'src'.
# Finalmente, imprimimos los primeros 1000 caracteres para visualizar una parte del contenido HTML recibido.

In [4]:
# Make a GET request
req = requests.get('https://www.ilga.gov/Senate/Members')
# Read the content of the server’s response
src = req.text
# View some output
print(src[:1000])

<!DOCTYPE html>
<html lang="en">
<head id="Head1">
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="content-type" content="text/html;charset=utf-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=Edge" />
    <meta charset="utf-8" />
    <meta charset="UTF-8">
    <!-- Meta Description -->
    <meta name="description" content="Welcome to the official government website of the Illinois General Assembly">
    <meta name="contactName" content="Legislative Information System">
    <meta name="contactOrganization" content="LIS Staff Services">
    <meta name="contactStreetAddress1" content="705 Stratton Office Building">
    <meta name="contactCity" content="Springfield">
    <meta name="contactZipcode" content="62706">
    <meta name="contactNetworkAddress" content="webmaster@ilga.gov">
    <meta name="contactPhoneNumber" content="217-782-3944">
    <meta name="contactFaxNumber" content="217-524-6059">
    <meta name


## Step 2: Parse the Page with Beautiful Soup

Now, we use the `BeautifulSoup` function to parse the reponse into an HTML tree. This returns an object (called a **soup object**) which contains all of the HTML in the original document.

If you run into an error about a parser library, make sure you've installed the `lxml` package to provide Beautiful Soup with the necessary parsing tools.

In [5]:
# Parse the response into an HTML tree
soup = BeautifulSoup(src, 'lxml')
# Take a look
print(soup.prettify()[:1000])

<!DOCTYPE html>
<html lang="en">
 <head id="Head1">
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta charset="utf-8"/>
  <meta charset="utf-8"/>
  <!-- Meta Description -->
  <meta content="Welcome to the official government website of the Illinois General Assembly" name="description"/>
  <meta content="Legislative Information System" name="contactName"/>
  <meta content="LIS Staff Services" name="contactOrganization"/>
  <meta content="705 Stratton Office Building" name="contactStreetAddress1"/>
  <meta content="Springfield" name="contactCity"/>
  <meta content="62706" name="contactZipcode"/>
  <meta content="webmaster@ilga.gov" name="contactNetworkAddress"/>
  <meta content="217-782-3944" name="contactPhoneNumber"/>
  <meta content="217-524-6059" name="contactFaxNumber"/>
  <meta content="State Of Illinois" name="originatorJur

Se verifica que se esta extrayendo la informacion de la pagina web

The output looks pretty similar to the above, but now it's organized in a `soup` object which allows us to more easily traverse the page.

## Step 3: Search for HTML Elements

Beautiful Soup has a number of functions to find useful components on a page. Beautiful Soup lets you find elements by their:

1. HTML tags
2. HTML Attributes
3. CSS Selectors

Let's search first for **HTML tags**. 

The function `find_all` searches the `soup` tree to find all the elements with an a particular HTML tag, and returns all of those elements.

What does the example below do?

se remplaza tr por la variable a, que es donde se guarda la informacion en la pagina web, analiznado el html para determinar donde se esta guardando la informacion

In [6]:
# Find all elements with a certain tag
a_tags = soup.find_all("a")
print(a_tags[:1000])

[<a b-0yw6sxot5c="" class="dropdown-item" data-lang="en" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-us"></span> English
                            </a>, <a b-0yw6sxot5c="" class="dropdown-item" data-lang="af" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-za"></span> Afrikaans
                            </a>, <a b-0yw6sxot5c="" class="dropdown-item" data-lang="sq" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-al"></span> Albanian
                            </a>, <a b-0yw6sxot5c="" class="dropdown-item" data-lang="ar" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-ae"></span> Arabic
                            </a>, <a b-0yw6sxot5c="" class="dropdown-item" data-lang="hy" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-am"></span> Armenian
                            </a>, <a b-0yw6sxot5c="" class="dropdown-item" data-lang="az" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-az"></span> Azerbaijani
            

Because `find_all()` is the most popular method in the Beautiful Soup search API, you can use a shortcut for it. If you treat the BeautifulSoup object as though it were a function, then it’s the same as calling `find_all()` on that object. 

These two lines of code are equivalent:

In [ ]:
#Aquí se demuestran dos formas diferentes (pero equivalentes) de extraer 
#etiquetas <a> usando BeautifulSoup: find_all() y el acceso directo como
#función (soup("a")). Se imprime el primer resultado de ambas listas para
# comparar.

In [7]:
a_tags = soup.find_all("a")
a_tags_alt = soup("a")
print(a_tags[0])
print(a_tags_alt[0])

<a b-0yw6sxot5c="" class="dropdown-item" data-lang="en" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-us"></span> English
                            </a>
<a b-0yw6sxot5c="" class="dropdown-item" data-lang="en" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-us"></span> English
                            </a>


How many links did we obtain?

In [73]:
print(len(a_tags))

270


luego de correr los comandos se obtiene 270 registros

That's a lot! Many elements on a page will have the same HTML tag. For instance, if you search for everything with the `a` tag, you're likely to get more hits, many of which you might not want. Remember, the `a` tag defines a hyperlink, so you'll usually find many on any given page.

What if we wanted to search for HTML tags with certain attributes, such as particular CSS classes? 

We can do this by adding an additional argument to the `find_all`. In the example below, we are finding all the `a` tags, and then filtering those with `class_="sidemenu"`.

se realiza el cambio de clase a "notranslate" que es la clase que guarda la variable a donde esta la informacion

In [76]:
# Get only the 'a' tags in 'notranslate' class
side_menus = soup("a", class_="notranslate")
side_menus[:50]

[<a class="notranslate" href="/Senate/Members/Details/3312">Neil Anderson</a>,
 <a class="notranslate" href="/Senate/Members/Details/3312">Neil Anderson</a>,
 <a class="notranslate" href="/Senate/Members/Details/3316">Omar Aquino</a>,
 <a class="notranslate" href="/Senate/Members/Details/3316">Omar Aquino</a>,
 <a class="notranslate" href="/Senate/Members/Details/3383">Li Arellano, Jr.</a>,
 <a class="notranslate" href="/Senate/Members/Details/3383">Li Arellano, Jr.</a>,
 <a class="notranslate" href="/Senate/Members/Details/3413">Chris Balkema</a>,
 <a class="notranslate" href="/Senate/Members/Details/3413">Chris Balkema</a>,
 <a class="notranslate" href="/Senate/Members/Details/3337">Christopher Belt</a>,
 <a class="notranslate" href="/Senate/Members/Details/3337">Christopher Belt</a>,
 <a class="notranslate" href="/Senate/Members/Details/3386">Terri Bryant</a>,
 <a class="notranslate" href="/Senate/Members/Details/3386">Terri Bryant</a>,
 <a class="notranslate" href="/Senate/Members/

A more efficient way to search for elements on a website is via a **CSS selector**. For this we have to use a different method called `select()`. Just pass a string into the `.select()` to get all elements with that string as a valid CSS selector.

In the example above, we can use `"a.sidemenu"` as a CSS selector, which returns all `a` tags with class `sidemenu`.

In [78]:
# Get elements with "a.notranslate" CSS Selector.
selected = soup.select("a.notranslate")
selected[:50]

[<a class="notranslate" href="/Senate/Members/Details/3312">Neil Anderson</a>,
 <a class="notranslate" href="/Senate/Members/Details/3312">Neil Anderson</a>,
 <a class="notranslate" href="/Senate/Members/Details/3316">Omar Aquino</a>,
 <a class="notranslate" href="/Senate/Members/Details/3316">Omar Aquino</a>,
 <a class="notranslate" href="/Senate/Members/Details/3383">Li Arellano, Jr.</a>,
 <a class="notranslate" href="/Senate/Members/Details/3383">Li Arellano, Jr.</a>,
 <a class="notranslate" href="/Senate/Members/Details/3413">Chris Balkema</a>,
 <a class="notranslate" href="/Senate/Members/Details/3413">Chris Balkema</a>,
 <a class="notranslate" href="/Senate/Members/Details/3337">Christopher Belt</a>,
 <a class="notranslate" href="/Senate/Members/Details/3337">Christopher Belt</a>,
 <a class="notranslate" href="/Senate/Members/Details/3386">Terri Bryant</a>,
 <a class="notranslate" href="/Senate/Members/Details/3386">Terri Bryant</a>,
 <a class="notranslate" href="/Senate/Members/

Se obtiene la informacion de los senadores de Illinois

## 🥊 Challenge: Find All

Use BeautifulSoup to find all the `a` elements with class `notranslate`.

In [79]:
# YOUR CODE HERE
soup.select("a.notranslate")

[<a class="notranslate" href="/Senate/Members/Details/3312">Neil Anderson</a>,
 <a class="notranslate" href="/Senate/Members/Details/3312">Neil Anderson</a>,
 <a class="notranslate" href="/Senate/Members/Details/3316">Omar Aquino</a>,
 <a class="notranslate" href="/Senate/Members/Details/3316">Omar Aquino</a>,
 <a class="notranslate" href="/Senate/Members/Details/3383">Li Arellano, Jr.</a>,
 <a class="notranslate" href="/Senate/Members/Details/3383">Li Arellano, Jr.</a>,
 <a class="notranslate" href="/Senate/Members/Details/3413">Chris Balkema</a>,
 <a class="notranslate" href="/Senate/Members/Details/3413">Chris Balkema</a>,
 <a class="notranslate" href="/Senate/Members/Details/3337">Christopher Belt</a>,
 <a class="notranslate" href="/Senate/Members/Details/3337">Christopher Belt</a>,
 <a class="notranslate" href="/Senate/Members/Details/3386">Terri Bryant</a>,
 <a class="notranslate" href="/Senate/Members/Details/3386">Terri Bryant</a>,
 <a class="notranslate" href="/Senate/Members/

## Step 4: Get Attributes and Text of Elements

Once we identify elements, we want the access information in that element. Usually, this means two things:

1. Text
2. Attributes

Getting the text inside an element is easy. All we have to do is use the `text` member of a `tag` object:

In [80]:
# Get all sidemenu links as a list
side_menu_links = soup.select("a.notranslate")

# Examine the first link, if available
if side_menu_links:
	first_link = side_menu_links[0]
	print(first_link)
	# What class is this variable?
	print('Class: ', type(first_link))
else:
	print("No sidemenu links found.")

<a class="notranslate" href="/Senate/Members/Details/3312">Neil Anderson</a>
Class:  <class 'bs4.element.Tag'>


It's a Beautiful Soup tag! This means it has a `text` member:

In [81]:
# Get all sidemenu links as a list
side_menu_links = soup.select("a.notranslate")

# Examine the first link, if available
if side_menu_links:
	first_link = side_menu_links[0]
	print(first_link.text)
else:
	print("No sidemenu links found.")

Neil Anderson


Sometimes we want the value of certain attributes. This is particularly relevant for `a` tags, or links, where the `href` attribute tells us where the link goes.

💡 **Tip**: You can access a tag’s attributes by treating the tag like a dictionary:

In [82]:
print(first_link.get('href'))

/Senate/Members/Details/3312


## 🥊 Challenge: Extract specific attributes

Extract all `href` attributes for each `mainmenu` URL.

In [84]:
[link['href'] for link in soup.select("a.notranslate")]


['/Senate/Members/Details/3312',
 '/Senate/Members/Details/3312',
 '/Senate/Members/Details/3316',
 '/Senate/Members/Details/3316',
 '/Senate/Members/Details/3383',
 '/Senate/Members/Details/3383',
 '/Senate/Members/Details/3413',
 '/Senate/Members/Details/3413',
 '/Senate/Members/Details/3337',
 '/Senate/Members/Details/3337',
 '/Senate/Members/Details/3386',
 '/Senate/Members/Details/3386',
 '/Senate/Members/Details/3317',
 '/Senate/Members/Details/3317',
 '/Senate/Members/Details/3403',
 '/Senate/Members/Details/3403',
 '/Senate/Members/Details/3410',
 '/Senate/Members/Details/3410',
 '/Senate/Members/Details/3443',
 '/Senate/Members/Details/3443',
 '/Senate/Members/Details/3291',
 '/Senate/Members/Details/3291',
 '/Senate/Members/Details/3329',
 '/Senate/Members/Details/3329',
 '/Senate/Members/Details/3334',
 '/Senate/Members/Details/3334',
 '/Senate/Members/Details/3407',
 '/Senate/Members/Details/3407',
 '/Senate/Members/Details/3339',
 '/Senate/Members/Details/3339',
 '/Senate/

<a id='scrape'></a>

# Scraping the Illinois General Assembly

Believe it or not, those are really the fundamental tools you need to scrape a website. Once you spend more time familiarizing yourself with HTML and CSS, then it's simply a matter of understanding the structure of a particular website and intelligently applying the tools of Beautiful Soup and Python.

Let's apply these skills to scrape the [Illinois 98th General Assembly](http://www.ilga.gov/senate/default.asp?GA=98).

Specifically, our goal is to scrape information on each senator, including their name, district, and party.

## Scrape and Soup the Webpage

Let's scrape and parse the webpage, using the tools we learned in the previous section.

In [110]:
# Make a GET request
req = requests.get('https://www.ilga.gov/Senate/Members/List')
# Read the content of the server’s response
src = req.text
# Soup it
soup = BeautifulSoup(src, "lxml")

## Search for the Table Elements

Our goal is to obtain the elements in the table on the webpage. Remember: rows are identified by the `tr` tag. Let's use `find_all` to obtain these elements.

In [111]:
# Get all table row elements
rows = soup.find_all("tr")
len(rows)

62

⚠️ **Warning**: Keep in mind: `find_all` gets *all* the elements with the `tr` tag. We only want some of them. If we use the 'Inspect' function in Google Chrome and look carefully, then we can use some CSS selectors to get just the rows we're interested in. Specifically, we want the inner rows of the table:

In [114]:
# Returns every ‘a’ css selector in the page
rows = soup.select('tr')

for row in rows[:500]:
    print(row, '\n')

<tr>
<th class="sortable-column" onclick="window.location.href = '?sortField=name&amp;sortOrder=asc';">
<span>Senator</span>
<span class="sort-indicator">↓↑</span>
</th>
<th class="sortable-column" onclick="window.location.href = '?sortField=district&amp;sortOrder=asc';">
<span>District</span>
<span class="sort-indicator">↓↑</span>
</th>
<th class="sortable-column" onclick="window.location.href = '?sortField=party&amp;sortOrder=asc';">
<span>Party</span>
<span class="sort-indicator">↓↑</span>
</th>
</tr> 

<tr class="member-card-container" data-name="neil anderson">
<td scope="row"><a class="notranslate" href="/Senate/Members/Details/3312">Neil Anderson</a></td>
<td>47</td>
<td>R</td>
</tr> 

<tr class="member-card-container" data-name="omar aquino">
<td scope="row"><a class="notranslate" href="/Senate/Members/Details/3316">Omar Aquino</a></td>
<td>2</td>
<td>D</td>
</tr> 

<tr class="member-card-container" data-name="li arellano">
<td scope="row"><a class="notranslate" href="/Senate/M

It looks like we want everything after the first two rows. Let's work with a single row to start, and build our loop from there.

In [116]:
example_row = rows[50]
print(example_row.prettify())

<tr class="member-card-container" data-name="dave syverson">
 <td scope="row">
  <a class="notranslate" href="/Senate/Members/Details/3265">
   Dave Syverson
  </a>
 </td>
 <td>
  35
 </td>
 <td>
  R
 </td>
</tr>



Let's break this row down into its component cells/columns using the `select` method with CSS selectors. Looking closely at the HTML, there are a couple of ways we could do this.

* We could identify the cells by their tag `td`.
* We could use the the class name `.detail`.
* We could combine both and use the selector `td.detail`.

In [129]:
for cell in example_row.select('td'):
    print(cell)
print()

for cell in example_row.select('.detail'):
    print(cell)
print()

for cell in example_row.select('td.detail'):
    print(cell)
print()

<td scope="row"><a class="notranslate" href="/Senate/Members/Details/3265">Dave Syverson</a></td>
<td>35</td>
<td>R</td>





We can confirm that these are all the same.

In [131]:
# Only compare selectors that should return the same elements
assert example_row.select('td.detail') == example_row.select('.detail')

Let's use the selector `td.detail` to be as specific as possible.

In [132]:
# Select only those 'td' tags with class 'detail' 
detail_cells = example_row.select('td')
detail_cells

[<td scope="row"><a class="notranslate" href="/Senate/Members/Details/3265">Dave Syverson</a></td>,
 <td>35</td>,
 <td>R</td>]

Most of the time, we're interested in the actual **text** of a website, not its tags. Recall that to get the text of an HTML element, we use the `text` member:

In [133]:
# Keep only the text in each of those cells
row_data = [cell.text for cell in detail_cells]

print(row_data)

['Dave Syverson', '35', 'R']


Looks good! Now we just use our basic Python knowledge to get the elements of this list that we want. Remember, we want the senator's name, their district, and their party.

In [135]:
print(row_data[0]) # Name
print(row_data[1]) # District
print(row_data[2]) # Party

Dave Syverson
35
R


## Getting Rid of Junk Rows

We saw at the beginning that not all of the rows we got actually correspond to a senator. We'll need to do some cleaning before we can proceed forward. Take a look at some examples:

In [136]:
print('Row 0:\n', rows[0], '\n')
print('Row 1:\n', rows[1], '\n')
print('Last Row:\n', rows[-1])

Row 0:
 <tr>
<th class="sortable-column" onclick="window.location.href = '?sortField=name&amp;sortOrder=asc';">
<span>Senator</span>
<span class="sort-indicator">↓↑</span>
</th>
<th class="sortable-column" onclick="window.location.href = '?sortField=district&amp;sortOrder=asc';">
<span>District</span>
<span class="sort-indicator">↓↑</span>
</th>
<th class="sortable-column" onclick="window.location.href = '?sortField=party&amp;sortOrder=asc';">
<span>Party</span>
<span class="sort-indicator">↓↑</span>
</th>
</tr> 

Row 1:
 <tr class="member-card-container" data-name="neil anderson">
<td scope="row"><a class="notranslate" href="/Senate/Members/Details/3312">Neil Anderson</a></td>
<td>47</td>
<td>R</td>
</tr> 

Last Row:
 <tr class="member-card-container" data-name="dan mcconchie">
<td scope="row"><a class="notranslate" href="/Senate/Members/Details/3315">Dan McConchie</a></td>
<td>26</td>
<td>R</td>
</tr>


When we write our for loop, we only want it to apply to the relevant rows. So we'll need to filter out the irrelevant rows. The way to do this is to compare some of these to the rows we do want, see how they differ, and then formulate that in a conditional.

As you can imagine, there a lot of possible ways to do this, and it'll depend on the website. We'll show some here to give you an idea of how to do this.

In [139]:
# Bad rows
print(len(rows[0]))
print(len(rows[1]))

# Good rows
print(len(rows[2]))
print(len(rows[3]))

7
7
7
7


Perhaps good rows have a length of 5. Let's check:

In [141]:
good_rows = [row for row in rows if row.select('td')]

# Let's check some rows
print(good_rows[0], '\n')
print(good_rows[-2], '\n')
print(good_rows[-1])

<tr class="member-card-container" data-name="neil anderson">
<td scope="row"><a class="notranslate" href="/Senate/Members/Details/3312">Neil Anderson</a></td>
<td>47</td>
<td>R</td>
</tr> 

<tr class="member-card-container" data-name="craig wilcox">
<td scope="row"><a class="notranslate" href="/Senate/Members/Details/3336">Craig Wilcox</a></td>
<td>32</td>
<td>R</td>
</tr> 

<tr class="member-card-container" data-name="dan mcconchie">
<td scope="row"><a class="notranslate" href="/Senate/Members/Details/3315">Dan McConchie</a></td>
<td>26</td>
<td>R</td>
</tr>


We found a footer row in our list that we'd like to avoid. Let's try something else:

In [142]:
rows[2].select('td') 

[<td scope="row"><a class="notranslate" href="/Senate/Members/Details/3316">Omar Aquino</a></td>,
 <td>2</td>,
 <td>D</td>]

In [143]:
# Bad row
print(rows[-1].select('td'), '\n')

# Good row
print(rows[5].select('td'), '\n')

# How about this?
good_rows = [row for row in rows if row.select('td')]

print("Checking rows...\n")
print(good_rows[0], '\n')
print(good_rows[-1])

[<td scope="row"><a class="notranslate" href="/Senate/Members/Details/3315">Dan McConchie</a></td>, <td>26</td>, <td>R</td>] 

[<td scope="row"><a class="notranslate" href="/Senate/Members/Details/3337">Christopher Belt</a></td>, <td>57</td>, <td>D</td>] 

Checking rows...

<tr class="member-card-container" data-name="neil anderson">
<td scope="row"><a class="notranslate" href="/Senate/Members/Details/3312">Neil Anderson</a></td>
<td>47</td>
<td>R</td>
</tr> 

<tr class="member-card-container" data-name="dan mcconchie">
<td scope="row"><a class="notranslate" href="/Senate/Members/Details/3315">Dan McConchie</a></td>
<td>26</td>
<td>R</td>
</tr>


Looks like we found something that worked!

## Loop it All Together

Now that we've seen how to get the data we want from one row, as well as filter out the rows we don't want, let's put it all together into a loop.

In [145]:
# Define storage list
members = []

# Get rid of junk rows
valid_rows = [row for row in rows if row.select('td')]

# Loop through all rows
for row in valid_rows:
    # Select only those 'td' tags with class 'detail'
    detail_cells = row.select('td')
    # Keep only the text in each of those cells
    row_data = [cell.text for cell in detail_cells]
    # Collect information
    name = row_data[0]
    district = int(row_data[1])
    party = row_data[2]
    # Store in a tuple
    senator = (name, district, party)
    # Append to list
    members.append(senator)

In [146]:
# Should be 61
len(members)

60

Let's take a look at what we have in `members`.

In [147]:
print(members[:5])

[('Neil Anderson', 47, 'R'), ('Omar Aquino', 2, 'D'), ('Li Arellano, Jr.', 37, 'R'), ('Chris Balkema', 53, 'R'), ('Christopher Belt', 57, 'D')]


## 🥊  Challenge: Get `href` elements pointing to members' bills 

The code above retrieves information on:  

- the senator's name,
- their district number,
- and their party.

We now want to retrieve the URL for each senator's list of bills. Each URL will follow a specific format. 

The format for the list of bills for a given senator is:

`http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=[MEMBER_ID]&Primary=True`

to get something like:

`http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True`

in which `MEMBER_ID=1911`. 

You should be able to see that, unfortunately, `MEMBER_ID` is not currently something pulled out in our scraping code.

Your initial task is to modify the code above so that we also **retrieve the full URL which points to the corresponding page of primary-sponsored bills**, for each member, and return it along with their name, district, and party.

Tips: 

* To do this, you will want to get the appropriate anchor element (`<a>`) in each legislator's row of the table. You can again use the `.select()` method on the `row` object in the loop to do this — similar to the command that finds all of the `td.detail` cells in the row. Remember that we only want the link to the legislator's bills, not the committees or the legislator's profile page.
* The anchor elements' HTML will look like `<a href="/senate/Senator.asp/...">Bills</a>`. The string in the `href` attribute contains the **relative** link we are after. You can access an attribute of a BeatifulSoup `Tag` object the same way you access a Python dictionary: `anchor['attributeName']`. See the <a href="http://www.crummy.com/software/BeautifulSoup/bs4/doc/#tag">documentation</a> for more details.
* There are a _lot_ of different ways to use BeautifulSoup to get things done. whatever you need to do to pull the `href` out is fine.

The code has been partially filled out for you. Fill it in where it says `#YOUR CODE HERE`. Save the path into an object called `full_path`.

In [148]:
# Make a GET request
req = requests.get('https://www.ilga.gov/Senate/Members/List')
# Read the content of the server’s response
src = req.text
# Soup it
soup = BeautifulSoup(src, "lxml")
# Create empty list to store our data
members = []

# Returns every ‘tr tr tr’ css selector in the page
rows = soup.select('tr')
# Get rid of junk rows
rows = [row for row in rows if row.select('td')]

# Loop through all rows
for row in rows:
    # Select only those 'td' tags with class 'detail'
    detail_cells = row.select('td') 
    # Keep only the text in each of those cells
    row_data = [cell.text for cell in detail_cells]
    # Collect information
    name = row_data[0]
    district = int(row_data[1])
    party = row_data[2]

    # YOUR CODE HERE
    full_path = ''

    # Store in a tuple
    senator = (name, district, party, full_path)
    # Append to list
    members.append(senator)

In [149]:
# Uncomment to test 
members[:5]

[('Neil Anderson', 47, 'R', ''),
 ('Omar Aquino', 2, 'D', ''),
 ('Li Arellano, Jr.', 37, 'R', ''),
 ('Chris Balkema', 53, 'R', ''),
 ('Christopher Belt', 57, 'D', '')]

## 🥊  Challenge: Modularize Your Code

Turn the code above into a function that accepts a URL, scrapes the URL for its senators, and returns a list of tuples containing information about each senator. 

In [ ]:
# YOUR CODE HERE
def get_senate_member_links(url):
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "lxml")
    members = []
    table = soup.find("table")
    if table:
        for row in table.find_all("tr")[1:]:  # Skip header
            cells = row.find_all("td")
            if len(cells) > 0:
                link_tag = cells[0].find("a")
                if link_tag and link_tag.get("href"):
                    name = link_tag.text.strip()
                    profile_url = "https://www.ilga.gov/Senate/Members" + link_tag["href"]
                    members.append((name, profile_url))
    return members

Se extraen los link de los miembros del senado, se verifica que en los links hay un "/" adicional, quitando el adicional los links funcionan y redirigen a la infromacion.

In [171]:
# Test your code
url = 'https://www.ilga.gov/Senate/Members/List'
members = get_senate_member_links(url)
len(members)

print(f"Total miembros: {len(members)}")
for member in members:
    print(member)


Total miembros: 59
('Neil Anderson', 'https://www.ilga.gov//Senate/Members/Details/3312')
('Omar Aquino', 'https://www.ilga.gov//Senate/Members/Details/3316')
('Li Arellano, Jr.', 'https://www.ilga.gov//Senate/Members/Details/3383')
('Chris Balkema', 'https://www.ilga.gov//Senate/Members/Details/3413')
('Christopher Belt', 'https://www.ilga.gov//Senate/Members/Details/3337')
('Terri Bryant', 'https://www.ilga.gov//Senate/Members/Details/3386')
('Cristina Castro', 'https://www.ilga.gov//Senate/Members/Details/3317')
('Javier L. Cervantes', 'https://www.ilga.gov//Senate/Members/Details/3403')
('Andrew S. Chesney', 'https://www.ilga.gov//Senate/Members/Details/3410')
('Lakesia Collins', 'https://www.ilga.gov//Senate/Members/Details/3443')
('Bill Cunningham', 'https://www.ilga.gov//Senate/Members/Details/3291')
('John F. Curran', 'https://www.ilga.gov//Senate/Members/Details/3329')
('Donald P. DeWitte', 'https://www.ilga.gov//Senate/Members/Details/3334')
('Mary Edly-Allen', 'https://www.i

## 🥊 Take-home Challenge: Writing a Scraper Function

We want to scrape the webpages corresponding to bills sponsored by each bills.

Write a function called `get_bills(url)` to parse a given bills URL. This will involve:

  - requesting the URL using the <a href="http://docs.python-requests.org/en/latest/">`requests`</a> library
  - using the features of the `BeautifulSoup` library to find all of the `<td>` elements with the class `billlist`
  - return a _list_ of tuples, each with:
      - description (2nd column)
      - chamber (S or H) (3rd column)
      - the last action (4th column)
      - the last action date (5th column)
      
This function has been partially completed. Fill in the rest.

In [168]:
def get_bills(url):
    import requests
    from bs4 import BeautifulSoup
    src = requests.get(url).text
    soup = BeautifulSoup(src, "lxml")
    rows = soup.select('tr')
    bills = []
    for row in rows:
        cells = row.find_all('td', class_='billlist')
        if len(cells) >= 5:
            bill_id = cells[0].text.strip()
            description = cells[1].text.strip()
            chamber = cells[2].text.strip()
            last_action = cells[3].text.strip()
            last_action_date = cells[4].text.strip()
            bill = (bill_id, description, chamber, last_action, last_action_date)
            bills.append(bill)
    return bills

In [172]:
# Uncomment to test your code
test_url = members[0][1]
get_bills(test_url)[0:5]

[]

### Scrape All Bills

Finally, create a dictionary `bills_dict` which maps a district number (the key) onto a list of bills (the value) coming from that district. You can do this by looping over all of the senate members in `members_dict` and calling `get_bills()` for each of their associated bill URLs.

**NOTE:** please call the function `time.sleep(1)` for each iteration of the loop, so that we don't destroy the state's web site.

In [ ]:
# YOUR CODE HERE
#debido a los cambios de la pagina no se puede obtener la informacion que se solicitaba en el enunciado

In [ ]:
# Uncomment to test your code
# bills_dict[52]